## Possible Solution to the cold start problem 

In [1]:
import pandas as pd
import numpy as np

In [2]:
movie_titles = pd.read_csv('../../data/movies.csv')
ratings = pd.read_csv('../../data/ratings.csv')

In [3]:
movie_titles.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
data = ratings.merge(movie_titles, on='movieId')

In [6]:
data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [7]:
data.shape

(100836, 6)

Create a dataframe that gives us the average rating for every movie

In [8]:
average_ratings = pd.DataFrame(data.groupby('title')['rating'].mean())
average_ratings.head(10)

,rating
title,
'71 (2014),4.000000
'Hellboy': The Seeds of Creation (2004),4.000000
'Round Midnight (1986),3.500000
'Salem's Lot (2004),5.000000
'Til There Was You (1997),4.000000
'Tis the Season for Love (2015),1.500000
"'burbs, The (1989)",3.176471
'night Mother (1986),3.000000
(500) Days of Summer (2009),3.666667


Add the quantity of ratings that each movie recieved to our dataframe

In [9]:
average_ratings['Num Ratings'] = pd.DataFrame(data.groupby('title')['rating'].count())
average_ratings.head()

,rating,Num Ratings
title,,
'71 (2014),4.0,1
'Hellboy': The Seeds of Creation (2004),4.0,1
'Round Midnight (1986),3.5,2
'Salem's Lot (2004),5.0,1
'Til There Was You (1997),4.0,2


##### Create a ratings table
- The individual users make up the Index
- The movies are the columns
- The Values are the ratings recieved by that individual customer

In [10]:
user_ratings = data.pivot_table(index='userId',columns='title',values='rating')

In [11]:
user_ratings.head(15)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


There are so many movies in the world, most individuals are not expected to have rated or to even have watched every movie out there.

In [12]:
def new_user_recommendations(movie_name, n=6):
    
    # Find movies that have a high correlation with the movie that the user selected    
    correlations = user_ratings.corrwith(user_ratings[movie_name])
    
    # Build a new Dataframe that has 
    recommendations = pd.DataFrame(correlations,columns=['Correlation'])
    
    # Drop rows with missing ratings
    recommendations.dropna(inplace=True)
    
    # Join the Num Ratings columns so that customers can see how many ratings a movie recieved
    recommendations = recommendations.join(average_ratings['Num Ratings'])
    
    # Select movies with more than 100 ratings, and put the highest correlated movies at the top
    user_recs = recommendations[recommendations['Num Ratings']>100].sort_values('Correlation',ascending=False).reset_index()
    
    # merge this dataframe with our original movies dataframe
    user_recs = user_recs.merge(movie_titles,on='title')
    
    return user_recs.head(n)

The Above Function allows a new user that we do not have data on to input a movie title that they really enjoy, and then recieve a selected number of movies that are similar. We only recommend movies that have over a 100 ratings to new customers because we want to recommend a generally popular movie so that the customer is inclined to continue using our service.

In [13]:
new_user_recommendations('Aladdin (1992)', 10)

C:\Users\wharr\Anaconda3\envs\learn-env\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\wharr\Anaconda3\envs\learn-env\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,title,Correlation,Num Ratings,movieId,genres
0,Aladdin (1992),1.000000,183,588,Adventure|Animation|Children|Comedy|Musical
1,Toy Story (1995),0.611892,215,1,Adventure|Animation|Children|Comedy|Fantasy
2,"Lion King, The (1994)",0.591660,172,364,Adventure|Animation|Children|Drama|Musical|IMAX
3,Beauty and the Beast (1991),0.582186,146,595,Animation|Children|Fantasy|Musical|Romance|IMAX
4,"Truman Show, The (1998)",0.562311,125,1682,Comedy|Drama|Sci-Fi
5,Finding Nemo (2003),0.543285,141,6377,Adventure|Animation|Children|Comedy
6,Back to the Future (1985),0.531754,171,1270,Adventure|Comedy|Sci-Fi
7,Shrek (2001),0.498432,170,4306,Adventure|Animation|Children|Comedy|Fantasy|Ro...
8,Ferris Bueller's Day Off (1986),0.474351,109,2918,Comedy
9,Forrest Gump (1994),0.464268,329,356,Comedy|Drama|Romance|War


Aladdin is most correlated with itself so it appears at the top, then the next 5 movies are our recommendations

In [14]:
new_user_recommendations('Forrest Gump (1994)')

C:\Users\wharr\Anaconda3\envs\learn-env\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\wharr\Anaconda3\envs\learn-env\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,title,Correlation,Num Ratings,movieId,genres
0,Forrest Gump (1994),1.000000,329,356,Comedy|Drama|Romance|War
1,Good Will Hunting (1997),0.484042,141,1704,Drama|Romance
2,Aladdin (1992),0.464268,183,588,Adventure|Animation|Children|Comedy|Musical
3,American History X (1998),0.457287,129,2329,Crime|Drama
4,"Truman Show, The (1998)",0.432556,125,1682,Comedy|Drama|Sci-Fi
5,Braveheart (1995),0.416976,237,110,Action|Drama|War


If the customer does not select a number of reviews they will recieve a default of 5 recommendations.